In [1]:
from pyspark.sql import SparkSession

In [2]:
import xml.etree.ElementTree as ET
import requests
import os
import collections
import time

import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()

Username id, type of content, id of content, rating (0,10), date of rating

Table of content(platform dependant)
Table of username (platform dependant)

In [4]:
def xml_to_dataframe(xml_file) -> pd.DataFrame:
    with open(xml_file, 'r') as f:
        r_text = f.read()
        root = ET.fromstring(r_text)

    df_user_id = []
    df_type = []
    df_content_id = []
    df_rating = []
    df_rating_date = []
    
    for bg in root:
        bg_name = bg[0].text
        coll_id = bg.attrib['collid']  # I don't really know what this is, but I guess it is the id of this instance of the boardgame in the list
        object_id = bg.attrib['objectid']  # This is the boardgame identifier
    
        for field in bg:
            if field.tag == 'stats':
                rating_val = field[0].attrib['value']
                if rating_val == 'N/A':
                    rating_val = None
            if field.tag == 'yearpublished':
                year_published = field.text
            if field.tag == 'status':
                date_of_rating = field.attrib['lastmodified']  # Not really the rating date, but it is as close as possible with the current information.

        # print(user_id, 'boardgame', object_id, rating_val, date_of_rating)
        # print(bg_name, rating_val, year_published, coll_id, object_id)
        df_user_id.append(xml.split('/')[-1][:-4])
        df_type.append('boardgame')
        df_content_id.append(object_id)
        df_rating.append(rating_val)
        df_rating_date.append(date_of_rating)

    return pd.DataFrame({
        'user_id': pd.Series(df_user_id, dtype='str'),
        'type': pd.Series(df_type, dtype='category'),
        'content_id': pd.Series(df_content_id, dtype='str'),
        'rating': pd.Series(df_rating, dtype='float64'),
        'rating_date': pd.Series(df_rating_date, dtype='datetime64[ms]')
    })

# user_id = 'eekspider'
# user_id = 'adammathys'
# xml_file = f'./boardgame/raw_data/collection/{user_id}.xml'
# xml_to_dataframe(xml_file)

In [5]:
base_xml_path = './boardgame/raw_data/collection'
base_parquet_path = './boardgame/parsed_data/collection'
for xml in filter(lambda x: x.endswith('.xml'), os.listdir(base_xml_path)):
    
    try:
        parquet_path = f'{base_parquet_path}/{xml[:-4]}.parquet'
        df = xml_to_dataframe(f'{base_xml_path}/{xml}')
        df.to_parquet(parquet_path)
    except:
        print(f'Error: Invalid xml file: {xml}')

Error: Invalid xml file: edou1289.xml
Error: Invalid xml file: kiminoboku.xml
Error: Invalid xml file: Roycannaday.xml
Error: Invalid xml file: Tverezyi_viking.xml
Error: Invalid xml file: Cooper10.xml
Error: Invalid xml file: siemlac.xml


# Spark

In [6]:
spark.read.parquet(base_parquet_path).show(10000)

+------------+---------+----------+------+-------------------+
|     user_id|     type|content_id|rating|        rating_date|
+------------+---------+----------+------+-------------------+
| zefquaavius|boardgame|    322232|   6.0|2023-08-01 14:52:32|
| zefquaavius|boardgame|    296402|   8.0|2023-08-02 14:18:24|
| zefquaavius|boardgame|    336537|  NULL|2023-08-02 14:18:38|
| zefquaavius|boardgame|    314445|  NULL|2023-08-02 14:18:54|
| zefquaavius|boardgame|    296404|  NULL|2023-08-02 14:19:11|
| zefquaavius|boardgame|    296406|  NULL|2023-08-01 14:56:42|
| zefquaavius|boardgame|    322429|  NULL|2023-08-02 14:19:32|
| zefquaavius|boardgame|    309782|   6.0|2023-08-01 14:57:01|
| zefquaavius|boardgame|    314446|  NULL|2023-08-02 14:19:57|
| zefquaavius|boardgame|    296407|  NULL|2023-08-01 14:57:31|
| zefquaavius|boardgame|    296423|  NULL|2023-08-01 14:57:46|
| zefquaavius|boardgame|    296411|  NULL|2023-08-02 14:20:11|
| zefquaavius|boardgame|    321710|  NULL|2023-08-02 14